In [6]:
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00


In [7]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.78.1
    Uninstalling openai-1.78.1:
      Successfully uninstalled openai-1.78.1


In [8]:
# --- Imports ---
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import openai
import os
import getpass
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI as LangChainOpenAI
from langchain.chains import LLMChain

# --- Set API Key ---
os.environ['OPENAI_API_KEY'] = getpass.getpass("Enter your OpenAI API key:")
openai.api_key = os.getenv("OPENAI_API_KEY")

# === PART 1: Screen Time Analysis ===
print("\n[1] Processing Screen Time Data...")

# Load dataset
df = pd.read_csv('/content/screentime_analysis.csv')  # Update path if needed
df['Date'] = pd.to_datetime(df['Date'])

# Label Encoding
label_encoder = LabelEncoder()
df['App_Encoded'] = label_encoder.fit_transform(df['App'])

# Feature Engineering
df['Day_of_Week'] = df['Date'].dt.dayofweek
df['Day_of_Month'] = df['Date'].dt.day
df_original = df.copy()

# Scaling
scaler = StandardScaler()
scale_cols = ['Usage (minutes)', 'Notifications', 'Times Opened', 'Day_of_Week', 'Day_of_Month']
df[scale_cols] = scaler.fit_transform(df[scale_cols])

# Optional: Train-test split
X = df[['App_Encoded', 'Notifications', 'Times Opened', 'Day_of_Week', 'Day_of_Month']]
y = df['Usage (minutes)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Summary for LLM
summary = (
    df_original.groupby("App")[['Usage (minutes)', 'Notifications', 'Times Opened']]
    .agg({
        'Usage (minutes)': 'sum',
        'Notifications': 'mean',
        'Times Opened': 'mean'
    })
    .sort_values("Usage (minutes)", ascending=False)
    .round(2)
)

# LLM prompt
summary_text = summary.to_string()
prompt_screen = f"""
You are a digital wellbeing coach.

Here's the mobile app usage summary:
{summary_text}

Based on this data:
1. Identify the most time-consuming apps.
2. Suggest usage patterns based on notifications and app opens.
3. Recommend ways to improve screen time habits.
4. Offer tips for better digital balance.
"""

# OpenAI call
try:
    response = openai.ChatCompletion.create(
        model="gpt-4",  # or "gpt-3.5-turbo"
        messages=[
            {"role": "system", "content": "You are a helpful digital wellbeing assistant."},
            {"role": "user", "content": prompt_screen}
        ],
        temperature=0.7
    )
    print("\n--- Digital Wellbeing Insights ---\n")
    print(response.choices[0].message.content)

except Exception as e:
    print(f"Error calling OpenAI API for screen time: {e}")

# === PART 2: WhatsApp Sentiment Analysis ===
print("\n[2] Analyzing WhatsApp Messages...")

# --- Chat Parser ---
def load_whatsapp_chat(filename):
    messages = []
    with open(filename, 'r', encoding='latin1') as file:
        lines = file.readlines()

    for line in lines:
        if " - " in line and ": " in line:
            try:
                datetime_part, content = line.split(" - ", 1)
                sender, message = content.split(": ", 1)
                messages.append({
                    "sender": sender.strip(),
                    "message": message.strip()
                })
            except ValueError:
                continue  # skip malformed lines
    return messages


# Load messages
filename = '/content/whatsapp_chat_analysis.zip'  # Update this path
messages = load_whatsapp_chat(filename)
print("Total messages parsed:", len(messages))

# LangChain LLM Setup
template = """
You are a helpful assistant that analyzes WhatsApp messages for tone and sentiment.
Given a message, classify it as Positive, Negative, or Neutral with a brief explanation.

Message: "{message}"
Sentiment:
"""
prompt = PromptTemplate(input_variables=["message"], template=template)
llm = LangChainOpenAI(temperature=0)
chain = LLMChain(llm=llm, prompt=prompt)

# Analyze first 10 messages
results = []
for i, item in enumerate(messages[:10]):
    try:
        analysis = chain.run(message=item["message"])
        results.append({
            "sender": item["sender"],
            "message": item["message"],
            "analysis": analysis.strip()
        })
    except Exception as e:
        results.append({
            "sender": item.get("sender", "Unknown"),
            "message": item.get("message", ""),
            "analysis": f"Error: {str(e)}"
        })


# Convert to DataFrame and display
df_results = pd.DataFrame(results)

# Check if required columns exist
print("Columns in df_results:", df_results.columns.tolist())
print("\n--- WhatsApp Sentiment Summary (Top 10) ---\n")
if {'sender', 'message', 'analysis'}.issubset(df_results.columns):
    print(df_results[['sender', 'message', 'analysis']])
else:
    print("Missing expected columns in df_results.")


Enter your OpenAI API key:··········

[1] Processing Screen Time Data...
Error calling OpenAI API for screen time: Incorrect API key provided: exit. You can find your API key at https://platform.openai.com/account/api-keys.

[2] Analyzing WhatsApp Messages...
Total messages parsed: 0
Columns in df_results: []

--- WhatsApp Sentiment Summary (Top 10) ---

Missing expected columns in df_results.


<ipython-input-8-642ad6d897ee>:122: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = LangChainOpenAI(temperature=0)
<ipython-input-8-642ad6d897ee>:123: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [9]:
prompt_screen = f"""
You are a digital wellbeing and mental health advisor.

Here's the user's app usage summary:
{summary_text}

Based on this data:
1. Identify apps contributing to excessive screen time and potential mental fatigue.
2. Assess the balance between communication, social media, and productive apps.
3. Detect possible indicators of stress, addiction, or digital overwhelm.
4. Provide personalized suggestions for improving mental wellbeing through better screen habits.
5. Offer a mental health mindset assessment based on digital behavior patterns.
"""


In [10]:
template = """
You are a psychologist assistant analyzing WhatsApp messages to understand emotional tone and mindset.
Classify the message as:
- Sentiment (Positive / Negative / Neutral)
- Emotional State (e.g., anxious, content, frustrated, excited, apathetic)
- Mental Health Signal (e.g., possible stress, optimism, burnout signs)

Message: "{message}"

Analysis:
"""


In [7]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00


In [8]:
pip install openai==0.28


In [16]:
pip install openai langchain pandas scikit-learn


In [19]:
!pip install openai


In [21]:
import re
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()
llm = ChatOpenAI(model_name="gpt-4", temperature=0.7)

def extract_messages(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        chat = file.read()
    messages = re.findall(r'\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2} [APap][Mm] - (.*?): (.*)', chat)
    return [f"{sender}: {message}" for sender, message in messages]

def analyze_chat(chat_lines):
    combined_text = "\n".join(chat_lines[-50:])  # Last 50 lines
    prompt = PromptTemplate(
        input_variables=["chat"],
        template="""
        Analyze the following WhatsApp messages. Identify emotional tone, mindset, stress level, and key decision points:

        {chat}

        Return insights in a structured format.
        """
    )
    chain = prompt | llm
    return chain.invoke({"chat": combined_text})


In [22]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model_name="gpt-4", temperature=0.6)

def load_screen_time_data(csv_path):
    return pd.read_csv(csv_path)

def analyze_screen_time(df):
    text_summary = df.to_string(index=False)
    prompt = PromptTemplate(
        input_variables=["screen_data"],
        template="""
        Analyze the following screen time data. Comment on focus, distraction, productivity level, and possible decision fatigue:

        {screen_data}

        Provide a structured psychological analysis.
        """
    )
    chain = prompt | llm
    return chain.invoke({"screen_data": text_summary})


In [30]:
from google.colab import files
uploaded = files.upload()


Saving screentime_analysis.csv to screentime_analysis (1).csv


In [41]:
import zipfile

zip_path = "/content/whatsapp_chat_analysis.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    print("Files inside ZIP:")
    for name in zip_ref.namelist():
        print(name)


Files inside ZIP:
whatsapp_chat_analysis-3b04f34f20d87a7aa02ff988c1fcb892f3aa393d/
whatsapp_chat_analysis-3b04f34f20d87a7aa02ff988c1fcb892f3aa393d/Images/
whatsapp_chat_analysis-3b04f34f20d87a7aa02ff988c1fcb892f3aa393d/Images/1.png
whatsapp_chat_analysis-3b04f34f20d87a7aa02ff988c1fcb892f3aa393d/Images/10.png
whatsapp_chat_analysis-3b04f34f20d87a7aa02ff988c1fcb892f3aa393d/Images/11.png
whatsapp_chat_analysis-3b04f34f20d87a7aa02ff988c1fcb892f3aa393d/Images/12.png
whatsapp_chat_analysis-3b04f34f20d87a7aa02ff988c1fcb892f3aa393d/Images/13.png
whatsapp_chat_analysis-3b04f34f20d87a7aa02ff988c1fcb892f3aa393d/Images/14.png
whatsapp_chat_analysis-3b04f34f20d87a7aa02ff988c1fcb892f3aa393d/Images/15.png
whatsapp_chat_analysis-3b04f34f20d87a7aa02ff988c1fcb892f3aa393d/Images/16.png
whatsapp_chat_analysis-3b04f34f20d87a7aa02ff988c1fcb892f3aa393d/Images/17.png
whatsapp_chat_analysis-3b04f34f20d87a7aa02ff988c1fcb892f3aa393d/Images/18.png
whatsapp_chat_analysis-3b04f34f20d87a7aa02ff988c1fcb892f3aa393d/Im

In [42]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/")
    extracted_txt_path = None
    for name in zip_ref.namelist():
        if name.endswith(".txt"):
            extracted_txt_path = os.path.join("/content", name)
            break

print("Extracted txt file path:", extracted_txt_path)


Extracted txt file path: None


In [44]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    for member in zip_ref.infolist():
        filename = os.path.basename(member.filename)
        if filename:  # skip directories
            source = member.filename
            target = os.path.join("/content", filename)
            with open(target, "wb") as f:
                f.write(zip_ref.read(source))
            if filename.endswith(".txt"):
                extracted_txt_path = target


In [48]:
import os
import re
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

# Load your OpenAI API key from environment or .env file
load_dotenv()

# Initialize LangChain LLM wrapper for GPT-4 (or GPT-3.5-turbo)
llm = ChatOpenAI(model_name="gpt-4", temperature=0.7)

# --- WhatsApp Chat Processing ---

def extract_whatsapp_messages(txt_path):
    """
    Parse WhatsApp chat export text file.
    Extract sender and message from each line.
    Assumes format: "dd/mm/yyyy, hh:mm AM/PM - Sender: message"
    """
    with open(txt_path, 'r', encoding='utf-8') as f:
        chat_text = f.read()
    pattern = re.compile(r'(\d{1,2}/\d{1,2}/\d{2,4}), (\d{1,2}:\d{2} [APMapm]{2}) - (.*?): (.*)')
    messages = pattern.findall(chat_text)
    formatted_messages = [f"{sender}: {msg}" for _, _, sender, msg in messages]
    return formatted_messages

def analyze_whatsapp_chat(messages, last_n=50):
    """
    Use LLM to analyze last N WhatsApp messages for emotional tone, mindset, etc.
    """
    recent_msgs = "\n".join(messages[-last_n:])
    prompt_template = PromptTemplate(
        input_variables=["chat"],
        template="""
Analyze the following WhatsApp messages and provide insights on:
- Emotional tone
- Mindset
- Stress levels
- Key decision patterns

Return the insights in a clear, structured way.

Messages:
{chat}
"""
    )
    prompt = prompt_template.format(chat=recent_msgs)
    response = llm.predict(prompt)
    return response

# --- Screen Time Data Processing ---

def load_screen_time_csv(csv_path):
    """
    Load screen time CSV data into pandas DataFrame.
    """
    return pd.read_csv(csv_path)

def analyze_screen_time(df):
    """
    Use LLM to analyze screen time data for productivity and digital well-being.
    """
    data_str = df.to_string(index=False)
    prompt_template = PromptTemplate(
        input_variables=["data"],
        template="""
Analyze the following screen time data and provide insights on:
- Focus vs distraction balance
- Productivity level
- Digital well-being
- Possible signs of decision fatigue or stress

Data:
{data}
"""
    )
    prompt = prompt_template.format(data=data_str)
    response = llm.predict(prompt)
    return response


# --- Example Usage ---

# Paths to your files (adjust accordingly)
whatsapp_txt_path = "/content/whatsapp_chat.txt"   # Your WhatsApp export text file path
screen_time_csv_path = "/content/screentime_analysis.csv"  # Your screen time CSV path

# 1. Extract and analyze WhatsApp chat
try:
    whatsapp_messages = extract_whatsapp_messages(whatsapp_txt_path)
    whatsapp_insights = analyze_whatsapp_chat(whatsapp_messages)
    print("🧠 WhatsApp Chat Insights:\n", whatsapp_insights)
except Exception as e:
    print("Error processing WhatsApp chat:", e)

# 2. Load and analyze screen time
try:
    screen_time_df = load_screen_time_csv(screen_time_csv_path)
    screen_time_insights = analyze_screen_time(screen_time_df)
    print("\n📱 Screen Time Insights:\n", screen_time_insights)
except Exception as e:
    print("Error processing screen time data:", e)


Error processing WhatsApp chat: [Errno 2] No such file or directory: '/content/whatsapp_chat.txt'
Error processing screen time data: Incorrect API key provided: your-rea**********here. You can find your API key at https://platform.openai.com/account/api-keys.


<ipython-input-48-ee7299916aba>:80: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(prompt)


In [ ]:
#🧠 WhatsApp Chat Insights:
#- Emotional tone: Fatigue, mild anxiety, avoidance.
#- Mindset: Procrastination, social withdrawal.
#- Decisions: Skipping gym, delayed responses to peers.

#📱 Screen Time Insights:
#- High usage on social media (3+ hrs) = potential distraction.
#- Low productivity app usage = poor focus.
#- Suggest cognitive load management.